In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

from sklearn.feature_extraction.text import CountVectorizer


%matplotlib inline

In [2]:
path_to_data = '/mnt/storage/HRV/data_atfer_Nekit_skripts/'
selected_pp = pd.read_stata(path_to_data + 'selected_mod.dta')

In [3]:
# количество здоровых людей

deasis = np.array(selected_pp.T[57:92].values)
# deasis = np.vstack((selected_pp.T[57:58].values, selected_pp.T[61:91].values))
deasis[deasis == 'No'] = 1
deasis[deasis == 'Have had'] = 0
deasis[deasis == 'Have now'] = 0
deasis[deasis == ''] = 1
a = np.sum(deasis, axis = 0)
len(a[a == 35])

17

In [4]:
#выбираем нужные для теста gidn

iSelected = np.vstack((selected_pp.M8_4_19 == 'Have now', selected_pp.M8_4_19 == 'Have had')).any(axis = 0)
files = selected_pp[iSelected].GIDN.values
len(files)

837

In [5]:
str2y = {'No':0, 'Have now':1, 'Have had':1}
str2num = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5}
#перевод триграммы в число

def trigram2num (string):
    num = 0
    for i, char in enumerate(string):
        num += str2num[char]*(6**i)
    
    return num

In [6]:
#перемешиваем R-T интервалы внутри сигнала одного больного для всех больных и считаем характеристики. 
# Тест неслучайности триграммы в последовательности кардиоциклов из статьи воронцова

start = time.clock()
codegramm = {}
F_w = []
B_w = []
for i in range(200):
    print (i, 'exp', 'time', round(time.clock() - start))
    X = []
    for file_i, file_ in enumerate(files[:]): #800 signals ~ 10 min
        if os.path.isfile(path_to_data + "R-T_code/" + str(file_) + '.npy'):
#             print (file_i, file_)

            data = np.load(path_to_data + 'R-T_code/' + str(file_) + '.npy')
            np.random.shuffle(data.T)
            fi = 1.0/(len(data) - 3)

            R = data[0]
            T = data[1]

            a = np.arctan(R/T)
            dT = T[1:] - T[:-1]
            dR = R[1:] - R[:-1]
            da = a[1:] - a[:-1]
            codegramm_i = []
            for i, dt in enumerate(dT):
                if R[i] <= 0 or R[i+1] <= 0:
                    print ('RRRRRRRrR', i, R[i], markup[i], T[i])
                    continue
                elif dt > 0 and dR[i] > 0 and da[i] > 0:
                    codegramm_i.append('A')
                elif dt <= 0 and dR[i] <= 0 and da[i] > 0:
                    codegramm_i.append('B')
                elif dt <=0 and dR[i] > 0 and da[i] > 0:
                    codegramm_i.append('C')
                elif dt > 0 and dR[i] <= 0 and da[i] <= 0:
                    codegramm_i.append('D')
                elif dt > 0 and dR[i] > 0 and da[i] <= 0:
                    codegramm_i.append('E')
                elif dt <= 0 and dR[i] <= 0 and da[i] <= 0:
                    codegramm_i.append('F')
                elif T[i] ==  0 or T[i+1] == 0:
                    continue
                else:
                    print ('ups...', i, 'dt', dt, 'dR', dR[i], 'da', da[i])

            ngramm = CountVectorizer(analyzer='char', ngram_range=(3, 3))
            analyze = ngramm.build_analyzer()
            words = analyze(''.join(codegramm_i))

            X_i = np.zeros(6**3)
            for i in words:
                X_i[trigram2num(i)] += 1

            X_i = X_i/sum(X_i)
            X.append(X_i)
        
    X = np.array(X)
    F_w_i = np.mean(X, axis = 0)
    F_w.append(F_w_i)
    X[X < fi] = 0
    B_w_i = np.mean(X, axis = 0)
    B_w.append(B_w_i)
    
F_w = np.array(F_w)
B_W = np.array(B_w)   
       
print ('time', round(time.clock() - start))

0 exp time 0
1 exp time 586
2 exp time 1167
3 exp time 1747
4 exp time 2329
5 exp time 2909
6 exp time 3490
7 exp time 4071
8 exp time 4653
9 exp time 5234
10 exp time 5815
11 exp time 6396
12 exp time 6978
13 exp time 7560
14 exp time 8142
15 exp time 8723
16 exp time 9303
17 exp time 9885
18 exp time 10467
19 exp time 11049
20 exp time 11630
21 exp time 12211
22 exp time 12793
23 exp time 13373
24 exp time 13954
25 exp time 14535
26 exp time 15117
27 exp time 15699
28 exp time 16280
29 exp time 16861
30 exp time 17442
31 exp time 18023
32 exp time 18604
33 exp time 19185
34 exp time 19767
35 exp time 20347
36 exp time 20929
37 exp time 21509
38 exp time 22092
39 exp time 22672
40 exp time 23253
41 exp time 23835
42 exp time 24416
43 exp time 24998
44 exp time 25579
45 exp time 26160
46 exp time 26741
47 exp time 27323
48 exp time 27903
49 exp time 28485
50 exp time 29065
51 exp time 29645
52 exp time 30226
53 exp time 30808
54 exp time 31390
55 exp time 31972
56 exp time 32552
57 exp

In [7]:
# считает характеристики по исходным сигналам

X = []
for file_i, file_ in enumerate(files[:]):
    if os.path.isfile(path_to_data + "R-T_code/" + str(file_) + '.npy'):
#             print (file_i, file_)

        data = np.load(path_to_data + 'R-T_code/' + str(file_) + '.npy')
        fi = 1.0/(len(data) - 3)

        R = data[0]
        T = data[1]

        a = np.arctan(R/T)
        dT = T[1:] - T[:-1]
        dR = R[1:] - R[:-1]
        da = a[1:] - a[:-1]
        codegramm_i = []
        for i, dt in enumerate(dT):
            if R[i] <= 0 or R[i+1] <= 0:
                print ('RRRRRRRrR', i, R[i], markup[i], T[i])
                continue
            elif dt > 0 and dR[i] > 0 and da[i] > 0:
                codegramm_i.append('A')
            elif dt <= 0 and dR[i] <= 0 and da[i] > 0:
                codegramm_i.append('B')
            elif dt <=0 and dR[i] > 0 and da[i] > 0:
                codegramm_i.append('C')
            elif dt > 0 and dR[i] <= 0 and da[i] <= 0:
                codegramm_i.append('D')
            elif dt > 0 and dR[i] > 0 and da[i] <= 0:
                codegramm_i.append('E')
            elif dt <= 0 and dR[i] <= 0 and da[i] <= 0:
                codegramm_i.append('F')
            elif T[i] ==  0 or T[i+1] == 0:
                continue
            else:
                print ('ups...', i, 'dt', dt, 'dR', dR[i], 'da', da[i])

        ngramm = CountVectorizer(analyzer='char', ngram_range=(3, 3))
        analyze = ngramm.build_analyzer()
        words = analyze(''.join(codegramm_i))

        X_i = np.zeros(6**3)
        for i in words:
            X_i[trigram2num(i)] += 1

        X_i = X_i/sum(X_i)
        X.append(X_i)

X = np.array(X)
F_w_orig = np.mean(X, axis = 0)
X[X < fi] = 0
B_w_orig = np.mean(X, axis = 0)


In [8]:
#смотрим, какие триграммы несут информацию

res1 = np.vstack((B_w_orig - np.min(B_w, axis = 0) < 0, np.max(B_w, axis = 0) - B_w_orig < 0)).T.any(axis=1)

res2 = np.vstack((F_w_orig - np.min(F_w, axis = 0) < 0, np.max(F_w, axis = 0) - F_w_orig < 0)).T.any(axis=1)

DF = (2*F_w_orig - np.min(F_w, axis = 0) - np.max(F_w, axis = 0))/(np.max(F_w, axis = 0) - np.min(F_w, axis = 0))
res3 = np.vstack((DF > 1, DF < -1)).T.any(axis=1)

DB = (2*B_w_orig - np.min(B_w, axis = 0) - np.max(B_w, axis = 0))/(np.max(B_w, axis = 0) - np.min(B_w, axis = 0))
res4 = np.vstack((DB > 1, DB < -1)).T.any(axis=1)

res_all = np.vstack((res1, res2, res3, res4)).T.any(axis=1)
res_all

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,

In [9]:
#trigramm не содержащие информации [  7, 125, 148, 151, 162, 202] по тесту 1
#[119, 149]

np.array(range(2016))[~res_all]

array([119, 149])

In [10]:
np.save(path_to_data + 'bad_trigramm_1', res_all)

Второй тест на информативность. выбираем случайных людей из выборки больных и здоровых. и считаем их больными.

In [11]:
files = selected_pp[iSelected].GIDN.values


In [12]:
# Тест неслучайности триграммы в последовательности кардиоциклов из статьи воронцова

start = time.clock()
codegramm = {}
F_w = []
B_w = []
for i in range(400):
    print (i, 'exp', 'time', round(time.clock() - start))
    files_i = np.random.choice(files, 900)
    X = []
    for file_i, file_ in enumerate(files_i[:]): #800 signals ~ 10 min
        if os.path.isfile(path_to_data + "code_string/" + str(file_) + '.txt'):
#             print (file_i, file_)

            ngramm = CountVectorizer(input = 'file', analyzer='char', ngram_range=(3, 3))
            analyze = ngramm.build_analyzer()
            with open(path_to_data + 'code_string/' + str(file_) + '.txt', "r") as o:
                words = analyze(o)
                
            fi = 1.0/(len(words) - 1)

            X_i = np.zeros(6**3)
            for i in words:
                X_i[trigram2num(i)] += 1

            X_i = X_i/sum(X_i)
            X.append(X_i)
        
    X = np.array(X)
    F_w_i = np.mean(X, axis = 0)
    F_w.append(F_w_i)
    X[X < fi] = 0
    B_w_i = np.mean(X, axis = 0)
    B_w.append(B_w_i)
    
F_w = np.array(F_w)
B_W = np.array(B_w)   
       
print ('time', round(time.clock() - start))

0 exp time 0
1 exp time 150
2 exp time 301
3 exp time 450
4 exp time 601
5 exp time 751
6 exp time 901
7 exp time 1051
8 exp time 1204
9 exp time 1356
10 exp time 1507
11 exp time 1659
12 exp time 1811
13 exp time 1962
14 exp time 2111
15 exp time 2260
16 exp time 2410
17 exp time 2561
18 exp time 2711
19 exp time 2860
20 exp time 3009
21 exp time 3159
22 exp time 3307
23 exp time 3456
24 exp time 3606
25 exp time 3755
26 exp time 3904
27 exp time 4055
28 exp time 4208
29 exp time 4357
30 exp time 4508
31 exp time 4657
32 exp time 4808
33 exp time 4959
34 exp time 5110
35 exp time 5261
36 exp time 5411
37 exp time 5560
38 exp time 5709
39 exp time 5861
40 exp time 6011
41 exp time 6162
42 exp time 6310
43 exp time 6459
44 exp time 6606
45 exp time 6758
46 exp time 6906
47 exp time 7056
48 exp time 7208
49 exp time 7359
50 exp time 7508
51 exp time 7660
52 exp time 7809
53 exp time 7960
54 exp time 8112
55 exp time 8261
56 exp time 8410
57 exp time 8558
58 exp time 8706
59 exp time 8856

In [13]:
# считает характеристики по исходным сигналам

X = []
for file_i, file_ in enumerate(files[:]):
    if os.path.isfile(path_to_data + "R-T_code/" + str(file_) + '.npy'):
#             print (file_i, file_)

        data = np.load(path_to_data + 'R-T_code/' + str(file_) + '.npy')
        fi = 1.0/(len(data) - 3)

        R = data[0]
        T = data[1]

        a = np.arctan(R/T)
        dT = T[1:] - T[:-1]
        dR = R[1:] - R[:-1]
        da = a[1:] - a[:-1]
        codegramm_i = []
        for i, dt in enumerate(dT):
            if R[i] <= 0 or R[i+1] <= 0:
                print ('RRRRRRRrR', i, R[i], markup[i], T[i])
                continue
            elif dt > 0 and dR[i] > 0 and da[i] > 0:
                codegramm_i.append('A')
            elif dt <= 0 and dR[i] <= 0 and da[i] > 0:
                codegramm_i.append('B')
            elif dt <=0 and dR[i] > 0 and da[i] > 0:
                codegramm_i.append('C')
            elif dt > 0 and dR[i] <= 0 and da[i] <= 0:
                codegramm_i.append('D')
            elif dt > 0 and dR[i] > 0 and da[i] <= 0:
                codegramm_i.append('E')
            elif dt <= 0 and dR[i] <= 0 and da[i] <= 0:
                codegramm_i.append('F')
            elif T[i] ==  0 or T[i+1] == 0:
                continue
            else:
                print ('ups...', i, 'dt', dt, 'dR', dR[i], 'da', da[i])

        ngramm = CountVectorizer(analyzer='char', ngram_range=(3, 3))
        analyze = ngramm.build_analyzer()
        words = analyze(''.join(codegramm_i))

        X_i = np.zeros(6**3)
        for i in words:
            X_i[trigram2num(i)] += 1

        X_i = X_i/sum(X_i)
        X.append(X_i)

X = np.array(X)
F_w_orig = np.mean(X, axis = 0)
X[X < fi] = 0
B_w_orig = np.mean(X, axis = 0)


In [14]:
#смотрим, какие триграммы несут информацию

res1 = np.vstack((B_w_orig - np.min(B_w, axis = 0) < 0, np.max(B_w, axis = 0) - B_w_orig < 0)).T.any(axis=1)

res2 = np.vstack((F_w_orig - np.min(F_w, axis = 0) < 0, np.max(F_w, axis = 0) - F_w_orig < 0)).T.any(axis=1)

DF = (2*F_w_orig - np.min(F_w, axis = 0) - np.max(F_w, axis = 0))/(np.max(F_w, axis = 0) - np.min(F_w, axis = 0))
res3 = np.vstack((DF > 1, DF < -1)).T.any(axis=1)

DB = (2*B_w_orig - np.min(B_w, axis = 0) - np.max(B_w, axis = 0))/(np.max(B_w, axis = 0) - np.min(B_w, axis = 0))
res4 = np.vstack((DB > 1, DB < -1)).T.any(axis=1)

res_all = np.vstack((res1, res2, res3, res4)).T.any(axis=1)
res_all

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [15]:
#trigramm не содержащие информации  по тесту 2 [  0,  14,  17,  18,  27,  32,  33,  44,  50,  51,  66,  69,  73,
#         86,  89,  91,  99, 106, 107, 111, 123, 129, 143, 144, 148, 152,
#        155, 156, 161, 172, 173, 177, 189, 190, 191, 201, 202, 212, 213

np.array(range(2016))[~res_all]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [16]:
np.save(path_to_data + 'bad_trigramm_2', res_all)